Use the Nvidia GPU if possible

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Apr 28 22:35:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

****Connect google drive in which the dataset is stored****

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive


Mounted at /content/gdrive


**1) Clone the Darknet repository**



In [3]:
!git clone https://github.com/AlexeyAB/darknet


Cloning into 'darknet'...
remote: Enumerating objects: 14997, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 14997 (delta 18), reused 26 (delta 8), pack-reused 14951
Receiving objects: 100% (14997/14997), 13.38 MiB | 21.55 MiB/s, done.
Resolving deltas: 100% (10180/10180), done.


**2) Compile Darknet using Nvidia GPU**


For this we have to make some changes to the make file in the darknet


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

Configuration file is prepared and can be edited using the notepad

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

On lines 610, 696 and 783, the no. of classes are changed from 80 to 1 since we are working on only one class - Solar panel.


On lines 603, 689 and 776, the no. of filters are changed from 255 to 18.

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

Create a folder with name yolov3 

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


Train the model with the class name Solar panel as given during labelling the image


In [8]:
!echo "Solar panel" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

Download the pretrained darknet 53 model 


In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-28 23:07:06--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   105MB/s    in 1.5s    

2021-04-28 23:07:07 (105 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
#unzip the images.zip to data/obj folder
!unzip /mydrive/Datascience/yolov3/images.zip -d data/obj

Archive:  /mydrive/Datascience/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/image1.jpg     
  inflating: data/obj/image1.txt     
  inflating: data/obj/image10.jpg    
  inflating: data/obj/image10.txt    
  inflating: data/obj/image1000.jpg  
  inflating: data/obj/image1000.txt  
  inflating: data/obj/image101.jpg   
  inflating: data/obj/image101.txt   
  inflating: data/obj/image102.jpg   
  inflating: data/obj/image102.txt   
  inflating: data/obj/image103.jpg   
  inflating: data/obj/image103.txt   
  inflating: data/obj/image104.jpg   
  inflating: data/obj/image104.txt   
  inflating: data/obj/image105.jpg   
  inflating: data/obj/image105.txt   
  inflating: data/obj/image106.jpg   
  inflating: data/obj/image106.txt   
  inflating: data/obj/image107.jpg   
  inflating: data/obj/image107.txt   
  inflating: data/obj/image108.jpg   
  inflating: data/obj/image108.txt   
  inflating: data/obj/image109.jpg   
  inflating: data/obj/image109.txt   
 

Read the txt files one by one in an array

In [11]:
# Convert the class index on the .txt files. Since there is only one class, it's supposed to be class 0.

import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

16 0.425557 0.391484 0.062898 0.063187

['16', '0.425557', '0.391484', '0.062898', '0.063187']
0 data/obj/image169.txt
0 0.425557 0.391484 0.062898 0.063187
16 0.060510 0.622253 0.052548 0.063187

['16', '0.060510', '0.622253', '0.052548', '0.063187']
0 data/obj/image169.txt
0 0.060510 0.622253 0.052548 0.063187
16 0.110271 0.596841 0.048567 0.075549

['16', '0.110271', '0.596841', '0.048567', '0.075549']
0 data/obj/image169.txt
0 0.110271 0.596841 0.048567 0.075549
16 0.769904 0.526786 0.049363 0.045330

['16', '0.769904', '0.526786', '0.049363', '0.045330']
0 data/obj/image169.txt
0 0.769904 0.526786 0.049363 0.045330
16 0.797771 0.553571 0.052548 0.065934

['16', '0.797771', '0.553571', '0.052548', '0.065934']
0 data/obj/image169.txt
0 0.797771 0.553571 0.052548 0.065934
16 0.311306 0.823489 0.062102 0.089286

['16', '0.311306', '0.823489', '0.062102', '0.089286']
1 data/obj/image108.txt
0 0.311306 0.823489 0.062102 0.089286
16 0.600318 0.652473 0.039809 0.090659

['16', '0.600318',

Print the image list

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/image148.jpg', 'data/obj/image235.jpg', 'data/obj/image104.jpg', 'data/obj/image114.jpg', 'data/obj/image233.jpg', 'data/obj/image191.jpg', 'data/obj/image123.jpg', 'data/obj/image116.jpg', 'data/obj/image13.jpg', 'data/obj/image2.jpg', 'data/obj/image174.jpg', 'data/obj/image170.jpg', 'data/obj/image135.jpg', 'data/obj/image117.jpg', 'data/obj/image158.jpg', 'data/obj/image112.jpg', 'data/obj/image11.jpg', 'data/obj/image241.jpg', 'data/obj/image236.jpg', 'data/obj/image183.jpg', 'data/obj/image155.jpg', 'data/obj/image126.jpg', 'data/obj/image131.jpg', 'data/obj/image108.jpg', 'data/obj/image10.jpg', 'data/obj/image182.jpg', 'data/obj/image154.jpg', 'data/obj/image23.jpg', 'data/obj/image140.jpg', 'data/obj/image197.jpg', 'data/obj/image106.jpg', 'data/obj/image156.jpg', 'data/obj/image107.jpg', 'data/obj/image125.jpg', 'data/obj/image192.jpg', 'data/obj/image101.jpg', 'data/obj/image22.jpg', 'data/obj/image111.jpg', 'data/obj/image167.jpg', 'data/obj/image168.jpg', 'data/

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**


Each 100 iterations, the custom object detector is going to be updated and saved in Google drive inside the folder “yolov3”.

 “yolov3_training_last.weights” is the final output weight 

 “yolov3_training__1000.weights”, “yolov3_training_2000.weights” and so on are the backup weights made by the darknet the model for each 1000 iterations.

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Sho

**Testing the images**

To test the images


1. load the libraries

In [14]:
import cv2
import numpy as np
import glob
import random
from google.colab.patches import cv2_imshow
from IPython.display import Image
import IPython
import PIL.Image

 2. Load the saved weight and the configuration file and name the object class to be detected

In [15]:
# Load Yolo
net = cv2.dnn.readNet(r"/content/gdrive/MyDrive/yolov3/yolov3_training_last.weights", r"/content/gdrive/MyDrive/yolov3/yolov3_testing.cfg")

# Name of the labelled object which is to be detected 
classes = ["Solar panel"]




 3. Load the images  for testing

In [16]:
# Images path
images_path = glob.glob(r"data/obj/*.jpg")


glob.glob("data/obj/*.jpg")


layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3)) 


random.shuffle(images_path)

5. Detection program 

Confidence and Area of bounding box or the  approx. area of solar panel is printed above the image

In [24]:
# loop through all the images
for img_path in images_path:
    # Loading image
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                print("Confidence ", confidence)
                print("Area of bounding box, ie. approx. area of solar panel", x*y,"pixels")

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    # Blue color in BGR
    color = (255, 230, 0)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            #color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.putText(img, label, (x, y + 30), font, 1, color, 2)
           
    
    IPython.display.display(PIL.Image.fromarray(img))
    

Output hidden; open in https://colab.research.google.com to view.